In [ ]:
import json
from datetime import datetime, timedelta

json_input = open("profs.json").read()

def get_time_slots(start_str, end_str):
    start = datetime.strptime(start_str, "%H:%M")
    end = datetime.strptime(end_str, "%H:%M")
    slots = []
    current = start
    while current < end:
        slots.append(current.strftime("%H:%M"))
        current += timedelta(minutes=15)
    return slots

def calculate_rowspan(start_str, end_str):
    fmt = "%H:%M"
    t1 = datetime.strptime(start_str, fmt)
    t2 = datetime.strptime(end_str, fmt)
    diff = t2 - t1
    return int(diff.total_seconds() / 900)

def generate_html(data):
    html_content = """
    <html>
    <head>
        <style>
            body { font-family: 'Malgun Gothic', 'AppleGothic', sans-serif; padding: 20px; background: #f0f0f0; }
            .page { background: white; width: 210mm; min-height: 297mm; padding: 20px; margin: 0 auto 20px auto; box-shadow: 0 0 10px rgba(0,0,0,0.1); }
            h1 { text-align: center; text-decoration: underline; text-underline-offset: 5px; margin-bottom: 30px; }
            .header-info { margin-bottom: 10px; font-size: 14px; font-weight: bold; }
            table { width: 100%; border-collapse: collapse; table-layout: fixed; margin-bottom: 50px; }
            th, td { border: 1px solid black; text-align: center; font-size: 12px; vertical-align: middle; padding: 2px; height: 30px; }
            th { background-color: #d3d3d3; height: 40px; }
            .time-col { width: 80px; background-color: #f9f9f9; font-weight: bold; }
            .class-content { display: flex; flex-direction: column; justify-content: center; height: 100%; font-size: 11px; }
            .class-id { font-weight: bold; }
        </style>
    </head>
    <body>
    """

    all_time_slots = get_time_slots("08:00", "23:45")
    days_map = {"Mon": 0, "Tue": 1, "Wed": 2, "Thu": 3, "Fri": 4, "Sat": 5}
    parsed_data = json.loads(data)
    for timetable in parsed_data.get("timetables", []):
        prof_name = timetable.get("professor", "Unknown")
        html_content += f"""
        <div class="page">
            <h1>교 수 강 의 시 간 표</h1>
            <div class="header-info">
                교수 : {prof_name}
            </div>
            <table>
                <thead>
                    <tr>
                        <th class="time-col">시간</th>
                        <th>월</th><th>화</th><th>수</th><th>목</th><th>금</th><th>토</th>
                    </tr>
                </thead>
                <tbody>
        """
        grid_state = {t: [None]*6 for t in all_time_slots} 
        for cls in timetable.get("classes", []):
            rowspan = calculate_rowspan(cls['start_time'], cls['end_time'])
            display_text = f"""
            <div class='class-content'>
                <span class='class-id'>{cls['class_id']} {cls['section']}</span>
                <span>{cls['credit']} {cls['title']}</span>
                <span>{cls['building_room']}</span>
            </div>
            """
            for day_str in cls['day']:
                if day_str not in days_map: continue
                day_idx = days_map[day_str]
                start_time = cls['start_time']
                if start_time in grid_state:
                    grid_state[start_time][day_idx] = {"html": display_text, "span": rowspan}
                    current_idx = all_time_slots.index(start_time)
                    for i in range(1, rowspan):
                        if current_idx + i < len(all_time_slots):
                            next_time = all_time_slots[current_idx + i]
                            grid_state[next_time][day_idx] = {"html": None, "span": -1}
        for time in all_time_slots:
            html_content += f"<tr><td class='time-col'>{time}</td>"
            for day_idx in range(6):
                cell_data = grid_state[time][day_idx]
                if cell_data is None:
                    html_content += "<td></td>"
                elif cell_data["span"] == -1:
                    continue
                else:
                    html_content += f"<td rowspan='{cell_data['span']}'>{cell_data['html']}</td>"
            html_content += "</tr>"
        html_content += """
                </tbody>
            </table>
        </div>
        """
    html_content += "</body></html>"
    with open("reconstructed_timetable.html", "w", encoding="utf-8") as f:
        f.write(html_content)
generate_html(json_input)

In [ ]:
import json

raw_links = """
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyMSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 전상률
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyJTJGYXJ0Y2xWaWV3LmRvJTNG : 권동현
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxJTJGYXJ0Y2xWaWV3LmRvJTNG : 감진규
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxOSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 이도훈
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxMiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 손준영
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxMyUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 송길태
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0NCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 김길호
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0MCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 권혁철
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxNyUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 유영환
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxNiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 우균
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxNSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 염근혁
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxNCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 안성용
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxMSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 백윤주
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYxMCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 박진선
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY5JTJGYXJ0Y2xWaWV3LmRvJTNG : 박영진
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY4JTJGYXJ0Y2xWaWV3LmRvJTNG : 김호원
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY3JTJGYXJ0Y2xWaWV3LmRvJTNG : 김태운
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY2JTJGYXJ0Y2xWaWV3LmRvJTNG : 김종덕
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY1JTJGYXJ0Y2xWaWV3LmRvJTNG : 김정구
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0JTJGYXJ0Y2xWaWV3LmRvJTNG : 김원석
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0NiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 이선열
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyMCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 이명호
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0OCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 최동희
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyNCUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 채흥석
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyMyUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 조환규
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyMiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 조준수
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyNiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 탁성우
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyNSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 최윤호
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0MiUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 최성기
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkY0MyUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 한상곤
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkZjc2UlMkYyNyUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 황원주
https://cse.pusan.ac.kr/cse/75074/subview.do?enc=Zm5jdDF8QEB8JTJGcHJvSW50cm8lMkYzOSUyRmFydGNsVmlldy5kbyUzRg%3D%3D : 홍봉희
"""

prof_urls = {}
for line in raw_links.strip().split('\n'):
    if ':' in line:
        url, name = line.rsplit(':', 1)
        prof_urls[name.strip()] = url.strip()
prof_urls["이다영"] = ""
with open('profs.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
for timetable in data['timetables']:
    prof_name = timetable['professor']
    timetable['link'] = prof_urls.get(prof_name, "")
with open('profs_with_links.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [ ]:
import json
import difflib
import re

input_filename = 'profs.json'
with open(input_filename, 'r', encoding='utf-8') as f:
    profs_data = json.load(f)
actual_courses = set()
for timetable in profs_data['timetables']:
    for cls in timetable['classes']:
        actual_courses.add(cls['title'])
curriculum_courses = [
    "공학작문및발표", "컴퓨팅사고와인공지능", "기초컴퓨터프로그래밍", "공학미적분학",
    "이산수학", "컴퓨터및프로그래밍입문", "확률통계", "프로그래밍원리와실습",
    "인터넷과웹기초", "공학선형대수학", "C++프로그래밍과실습", "데이터과학입문",
    "인공지능수학", "자료구조", "인공지능개론", "AI프로그래밍", "컴퓨터알고리즘",
    "머신러닝", "인공지능세미나", "캡스톤디자인", "어드벤처디자인", "전기전자공학개론",
    "논리회로및설계", "시스템소프트웨어", "유닉스기초", "논리회로설계및실험",
    "웹응용프로그래밍", "컴퓨터구조", "플랫폼기반프로그래밍", "데이터통신",
    "실감미디어프로그래밍", "운영체제", "유닉스응용프로그래밍", "임베디드시스템",
    "컴퓨터비전개론", "그래프데이터분석", "데이터마이닝", "데이터베이스",
    "딥러닝프로그래밍", "소프트웨어공학", "인공지능융합설계및실험", "컴퓨터그래픽스",
    "컴퓨터네트워크", "가상현실개론", "빅데이터분석설계및실험", "산학협력실무",
    "생성모델", "시각컴퓨팅", "임베디드소프트웨어설계", "자연어처리", "지능형IoT플랫폼",
    "클라우드컴퓨팅", "프로그래밍언어론", "강화학습개론", "멀티모달딥러닝",
    "인간컴퓨터상호작용", "정보검색및추천시스템", "일반물리학", "소프트웨어설계및실험",
    "임베디드시스템설계및실험", "파일구조", "컴파일러", "소프트웨어시스템설계",
    "정보보안", "집적회로및시스템설계", "컴퓨터응용설계및실험", "블록체인",
    "네트워크보안", "멀티미디어처리", "사물인터넷", "대학영어", "고전읽기와토론"
]
pairing_result = {}
used_curriculum_items = set()

for actual in actual_courses:
    match = None
    clean_actual = re.sub(r'\([A-Z]{2,}\)$', '', actual).strip()
    if clean_actual in curriculum_courses:
        match = clean_actual
    if not match:
        for curr in curriculum_courses:
            if curr in clean_actual:
                match = curr
                break
    if not match:
        close_matches = difflib.get_close_matches(clean_actual, curriculum_courses, n=1, cutoff=0.6)
        if close_matches:
            match = close_matches[0]
    if match:
        pairing_result[actual] = match
        used_curriculum_items.add(match)
    else:
        pairing_result[actual] = "GUESS_IM_FORKED"
not_selected_curriculum = [c for c in curriculum_courses if c not in used_curriculum_items]
unmatched_actual = [k for k, v in pairing_result.items() if v == "GUESS_IM_FORKED"]

print(f"[NOT SELECTED] Curriculum Courses ({len(not_selected_curriculum)})")
for c in sorted(not_selected_curriculum):
    print(f" - {c}")
print(f"[UNMATCHED] Actual Courses ({len(unmatched_actual)})")
for c in sorted(unmatched_actual):
    print(f" - {c}")
output_filename = "course_pairs.json"
with open(output_filename, "w", encoding='utf-8') as f:
    json.dump(pairing_result, f, ensure_ascii=False, indent=2)

In [ ]:
import json

input_filename = 'profs.json'
with open(input_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)
unique_buildings = set()
unique_rooms = set()
for timetable in data['timetables']:
    for cls in timetable['classes']:
        room_code = cls.get('building_room', '').strip()
        if room_code:
            unique_rooms.add(room_code)
            if '-' in room_code:
                building_id = room_code.split('-')[0]
                unique_buildings.add(building_id)
            else:
                unique_buildings.add(room_code)
sorted_buildings = sorted(list(unique_buildings))
print(sorted_buildings)

In [ ]:
import json
import pandas as pd
import hashlib
import numpy as np
import re

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open('profs.json', 'r', encoding='utf-8') as f: profs = json.load(f)
with open('buildings.json', 'r', encoding='utf-8') as f: buildings = json.load(f)
dist_df = pd.read_csv('distance.csv', encoding='utf-8')
MAP_WIDTH = 849
MAP_HEIGHT = 728
OFFSET_X = 15
OFFSET_Y = 15
manual_pcts = {"102": (35.76, 80.80),"207": (57.88, 64.27),"310": (59.06, 56.00),"314": (70.94, 49.20),"704": (19.53, 24.40)}
for b_id, coords in buildings.items():
    if b_id in manual_pcts:
        coords['x_pct'] = manual_pcts[b_id][0]
        coords['y_pct'] = manual_pcts[b_id][1]
    else:
        adj_x = coords['x'] + OFFSET_X
        adj_y = coords['y'] + OFFSET_Y
        coords['x_pct'] = round((adj_x / MAP_WIDTH) * 100, 2)
        coords['y_pct'] = round((adj_y / MAP_HEIGHT) * 100, 2)

major_specs = {
    "공학작문및발표": {"AI": ("교양필수", "3-2"), "CS": ("교양필수", "3-2")},
    "컴퓨팅사고와인공지능": {"AI": ("교양필수", "1-1"), "CS": ("교양필수", "1-1")},
    "기초컴퓨터프로그래밍": {"AI": ("교양필수", "1-2"), "CS": ("교양필수", "1-2")},
    "공학미적분학": {"AI": ("기초교양", "1-1"), "CS": ("기초교양", "1-1")},
    "이산수학(I)": {"AI": ("일반선택", "N/A"), "CS": ("전공기초", "1-1")},
    "이산수학(II)": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "2-2")},
    "이산수학": {"AI": ("전공기초", "1-1"), "CS": ("일반선택", "N/A")},
    "일반물리학(I)": {"AI": ("일반선택", "N/A"), "CS": ("전공기초", "1-1")},
    "일반물리학(II)": {"AI": ("일반선택", "N/A"), "CS": ("전공기초", "1-2")},
    "컴퓨터및프로그래밍입문": {"AI": ("전공기초", "1-1"), "CS": ("전공기초", "1-1")},
    "확률통계": {"AI": ("전공기초", "1-2"), "CS": ("전공기초", "1-2")},
    "프로그래밍원리와실습": {"AI": ("전공기초", "1-2"), "CS": ("전공기초", "1-1,2")},
    "인터넷과웹기초": {"AI": ("전공필수", "1-1"), "CS": ("전공필수", "1-1")},
    "공학선형대수학": {"AI": ("전공필수", "1-2"), "CS": ("전공필수", "2-1")},
    "C++프로그래밍과실습": {"AI": ("전공필수", "2-1"), "CS": ("전공필수", "2-1")},
    "데이터과학입문": {"AI": ("전공필수", "2-1"), "CS": ("전공필수", "2-1")},
    "인공지능수학": {"AI": ("전공필수", "2-1"), "CS": ("일반선택", "N/A")},
    "자료구조": {"AI": ("전공필수", "2-2"), "CS": ("전공필수", "2-2")},
    "인공지능개론": {"AI": ("전공필수", "2-2"), "CS": ("전공선택", "3-2")},
    "AI프로그래밍": {"AI": ("전공필수", "2-2"), "CS": ("전공선택", "2-2")},
    "컴퓨터알고리즘": {"AI": ("전공필수", "3-1"), "CS": ("전공필수", "3-1")},
    "머신러닝": {"AI": ("전공필수", "3-1"), "CS": ("전공선택", "4-1")},
    "인공지능세미나특강": {"AI": ("전공필수", "3-2"), "CS": ("일반선택", "N/A")},
    "캡스톤디자인": {"AI": ("전공필수", "4-1,2"), "CS": ("전공필수", "4-1,2")},
    "어드벤처디자인": {"AI": ("전공선택", "1-2"), "CS": ("전공기초", "1-2")},
    "전기전자공학개론": {"AI": ("전공선택", "1-2"), "CS": ("전공기초", "1-2")},
    "논리회로및설계": {"AI": ("전공선택", "2-1"), "CS": ("전공필수", "2-1")},
    "시스템소프트웨어": {"AI": ("전공선택", "2-1"), "CS": ("전공선택", "2-1")},
    "유닉스기초": {"AI": ("전공선택", "2-1"), "CS": ("전공선택", "2-1")},
    "논리회로설계및실험": {"AI": ("전공선택", "2-2"), "CS": ("전공필수", "2-2")},
    "웹응용프로그래밍": {"AI": ("전공선택", "2-2"), "CS": ("전공선택", "2-2")},
    "컴퓨터구조": {"AI": ("전공선택", "2-2"), "CS": ("전공필수", "2-2")},
    "소프트웨어설계및실험": {"AI": ("일반선택", "N/A"), "CS": ("전공필수", "3-1")},
    "플랫폼기반프로그래밍": {"AI": ("전공선택", "2-2"), "CS": ("전공선택", "2-2")},
    "데이터통신": {"AI": ("전공선택", "3-1"), "CS": ("전공선택", "3-1")},
    "실감미디어프로그래밍": {"AI": ("전공선택", "3-1"), "CS": ("일반선택", "N/A")},
    "운영체제": {"AI": ("전공선택", "3-1"), "CS": ("전공필수", "3-1")},
    "유닉스응용프로그래밍": {"AI": ("전공선택", "3-1"), "CS": ("전공선택", "3-1")},
    "파일구조": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "3-1")},
    "임베디드시스템": {"AI": ("전공선택", "3-1"), "CS": ("전공선택", "3-1")},
    "컴퓨터비전개론": {"AI": ("전공선택", "3-1"), "CS": ("전공선택", "4-1")},
    "그래프데이터분석": {"AI": ("전공선택", "3-2"), "CS": ("일반선택", "N/A")},
    "데이터마이닝": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "4-2")},
    "데이터베이스": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "3-2")},
    "소프트웨어시스템설계": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-1")},
    "정보보안": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-1")},
    "집적회로및시스템설계": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-1")},
    "컴퓨터응용설계및실험": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-1")},
    "블록체인": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-1")},
    "네트워크보안": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-2")},
    "멀티미디어처리": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-2")},
    "사물인터넷": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "4-2")},
    "딥러닝프로그래밍": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "3-2")},
    "소프트웨어공학": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "3-2")},
    "인공지능융합설계및실험": {"AI": ("전공선택", "3-2"), "CS": ("일반선택", "N/A")},
    "컴퓨터그래픽스": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "3-2")},
    "컴퓨터네트워크": {"AI": ("전공선택", "3-2"), "CS": ("전공선택", "3-2")},
    "가상현실개론": {"AI": ("전공선택", "4-1"), "CS": ("일반선택", "N/A")},
    "빅데이터분석설계및실험": {"AI": ("전공선택", "4-1"), "CS": ("일반선택", "N/A")},
    "산학협력실무": {"AI": ("전공선택", "4-1,2"), "CS": ("전공선택", "4-1,2")},
    "생성모델": {"AI": ("전공선택", "4-1"), "CS": ("일반선택", "N/A")},
    "시각컴퓨팅": {"AI": ("전공선택", "4-1"), "CS": ("일반선택", "N/A")},
    "임베디드소프트웨어설계": {"AI": ("전공선택", "4-1"), "CS": ("전공선택", "4-1")},
    "임베디드시스템설계및실험": {"AI": ("일반선택", "N/A"), "CS": ("전공필수", "3-2")},
    "자연어처리": {"AI": ("전공선택", "4-1"), "CS": ("일반선택", "N/A")},
    "지능형IoT플랫폼": {"AI": ("전공선택", "4-1"), "CS": ("전공선택", "4-1")},
    "클라우드컴퓨팅": {"AI": ("전공선택", "4-1"), "CS": ("전공선택", "4-1")},
    "프로그래밍언어론": {"AI": ("전공선택", "4-1"), "CS": ("전공선택", "3-1")},
    "강화학습개론": {"AI": ("전공선택", "4-2"), "CS": ("일반선택", "N/A")},
    "멀티모달딥러닝": {"AI": ("전공선택", "4-2"), "CS": ("일반선택", "N/A")},
    "인간컴퓨터상호작용": {"AI": ("전공선택", "4-2"), "CS": ("일반선택", "N/A")},
    "정보검색및추천시스템": {"AI": ("전공선택", "4-2"), "CS": ("일반선택", "N/A")},
    "컴파일러": {"AI": ("일반선택", "N/A"), "CS": ("전공선택", "3-2")}
}

curriculum_whitelist = list(major_specs.keys())
def parse_time(t_str):
    if not t_str: return 0
    parts = t_str.split(':')
    return int(parts[0]) + int(parts[1])/60.0
color_map = {}
next_color_idx = 0
def get_color_idx(title):
    global next_color_idx
    if title not in color_map:
        color_map[title] = next_color_idx
        next_color_idx += 1
    return color_map[title]
catalog = {}
for canon_name in curriculum_whitelist:
    spec_data = major_specs.get(canon_name)
    formatted_specs = {"AI": {"type": spec_data["AI"][0], "year": spec_data["AI"][1]},"CS": {"type": spec_data["CS"][0], "year": spec_data["CS"][1]}}
    catalog[canon_name] = {"title": canon_name,"credit": 3.0,"specs": formatted_specs,"offerings": [],"isCurriculum": True,"colorIdx": get_color_idx(canon_name)}
for timetable in profs['timetables']:
    prof_name = timetable['professor']
    for cls in timetable['classes']:
        raw_title = cls['title']
        clean_name = raw_title.split('(')[0].strip()
        if raw_title in major_specs:
            canon_title = raw_title
        elif clean_name in major_specs:
            canon_title = clean_name
        else:
            canon_title = raw_title
        if canon_title not in catalog:
            catalog[canon_title] = {"title": canon_title,"credit": float(cls.get('credit', 3)),"specs": {"AI": {"type": "일반선택", "year": ""}, "CS": {"type": "일반선택", "year": ""}},"offerings": [],"isCurriculum": False,"colorIdx": get_color_idx(canon_title)}
        section_uid = f"{cls['class_id']}-{cls['section']}"
        existing = next((o for o in catalog[canon_title]['offerings'] if o['id'] == section_uid), None)
        parsed_times = []
        days = cls['day'] if isinstance(cls['day'], list) else [cls['day']]
        for d in days:
            parsed_times.append({"day": d,"start": parse_time(cls.get('start_time', '00:00')),"end": parse_time(cls.get('end_time', '00:00'))})
        raw_loc = cls.get('building_room', '')
        bldg_code = raw_loc.split('-')[0] if '-' in raw_loc else ""
        if raw_loc == "": bldg_code = "ONLINE"
        if existing:
            if prof_name not in existing['profs']: existing['profs'].append(prof_name)
            for pt in parsed_times:
                if not any(t['day'] == pt['day'] and t['start'] == pt['start'] for t in existing['times']):
                    existing['times'].append(pt)
            if existing['loc'] == "" and raw_loc != "":
                existing['loc'] = raw_loc
                existing['bldg'] = bldg_code
        else:
            catalog[canon_title]['offerings'].append({"id": section_uid,"profs": [prof_name],"times": parsed_times,"loc": raw_loc,"bldg": bldg_code,"section": cls.get('section', '')})

dist_map = {f"{r['a']}-{r['b']}": r['time'] for _, r in dist_df.iterrows()}
dist_map.update({f"{r['b']}-{r['a']}": r['time'] for _, r in dist_df.iterrows()})
final_db = {"courses": catalog, "buildings": buildings, "distances": dist_map}
final_db['buildings']['ONLINE'] = {"x": 0, "y": 0, "x_pct": 88, "y_pct": 12, "name": "Virtual Campus"}
with open('app_data.js', 'w', encoding='utf-8') as f:
    f.write(f"window.APP_DATA = {json.dumps(final_db, ensure_ascii=False, cls=NpEncoder)};")